In [1]:
# Essential imports
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import VGG19, ResNet50, Xception
from tensorflow.keras.applications.vgg19 import preprocess_input as vgg19_preprocess
from tensorflow.keras.applications.resnet import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.xception import preprocess_input as xception_preprocess
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import image_dataset_from_directory
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")


TensorFlow version: 2.18.1


In [2]:
# Dataset configuration
train_dir = 'acne image/train'
val_dir = 'acne image/val'
test_dir = 'acne image/test'

# Model parameters
batch_size = 32
epochs = 50
num_classes = 1  # Adjust based on your dataset

# Define image sizes for each model
vgg19_img_size = (224, 224)
resnet50_img_size = (224, 224)
xception_img_size = (299, 299)


In [3]:
!nvidia-smi

Fri Jul 11 00:03:45 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.57                 Driver Version: 576.57         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   52C    P0             12W /  105W |     769MiB /   8188MiB |     24%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
def create_data_generators_vgg19(train_dir, val_dir, test_dir, batch_size):
    """Create data generators for VGG19 with specific preprocessing"""
    
    # Training data generator with augmentation
    train_datagen = ImageDataGenerator(
        preprocessing_function=vgg19_preprocess,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        zoom_range=0.2,
        shear_range=0.2,
        fill_mode='nearest'
    )
    
    # Validation and test generators (no augmentation)
    val_test_datagen = ImageDataGenerator(
        preprocessing_function=vgg19_preprocess
    )
    
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=vgg19_img_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True,
        seed=42
    )
    
    val_generator = val_test_datagen.flow_from_directory(
        val_dir,
        target_size=vgg19_img_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )
    
    test_generator = val_test_datagen.flow_from_directory(
        test_dir,
        target_size=vgg19_img_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )
    
    return train_generator, val_generator, test_generator

def create_data_generators_resnet50(train_dir, val_dir, test_dir, batch_size):
    """Create data generators for ResNet50 with specific preprocessing"""
    
    # Training data generator with augmentation
    train_datagen = ImageDataGenerator(
        preprocessing_function=resnet_preprocess,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        zoom_range=0.2,
        shear_range=0.2,
        fill_mode='nearest'
    )
    
    # Validation and test generators (no augmentation)
    val_test_datagen = ImageDataGenerator(
        preprocessing_function=resnet_preprocess
    )
    
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=resnet50_img_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True,
        seed=42
    )
    
    val_generator = val_test_datagen.flow_from_directory(
        val_dir,
        target_size=resnet50_img_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )
    
    test_generator = val_test_datagen.flow_from_directory(
        test_dir,
        target_size=resnet50_img_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )
    
    return train_generator, val_generator, test_generator

def create_data_generators_xception(train_dir, val_dir, test_dir, batch_size):
    """Create data generators for Xception with specific preprocessing"""
    
    # Training data generator with augmentation
    train_datagen = ImageDataGenerator(
        preprocessing_function=xception_preprocess,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        zoom_range=0.2,
        shear_range=0.2,
        fill_mode='nearest'
    )
    
    # Validation and test generators (no augmentation)
    val_test_datagen = ImageDataGenerator(
        preprocessing_function=xception_preprocess
    )
    
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=xception_img_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True,
        seed=42
    )
    
    val_generator = val_test_datagen.flow_from_directory(
        val_dir,
        target_size=xception_img_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )
    
    test_generator = val_test_datagen.flow_from_directory(
        test_dir,
        target_size=xception_img_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )
    
    return train_generator, val_generator, test_generator


In [5]:
def create_vgg19_model(num_classes, input_shape=(224, 224, 3)):
    """Create VGG19 model for transfer learning"""
    
    # Load pre-trained VGG19 model
    base_model = VGG19(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    
    # Freeze base model layers
    base_model.trainable = False
    
    # Build the complete model
    inputs = layers.Input(shape=input_shape)
    x = vgg19_preprocess(inputs)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = models.Model(inputs, outputs)
    return model, base_model

def create_resnet50_model(num_classes, input_shape=(224, 224, 3)):
    """Create ResNet50 model for transfer learning"""
    
    # Load pre-trained ResNet50 model
    base_model = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    
    # Freeze base model layers
    base_model.trainable = False
    
    # Build the complete model
    inputs = layers.Input(shape=input_shape)
    x = resnet_preprocess(inputs)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = models.Model(inputs, outputs)
    return model, base_model

def create_xception_model(num_classes, input_shape=(299, 299, 3)):
    """Create Xception model for transfer learning"""
    
    # Load pre-trained Xception model
    base_model = Xception(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    
    # Freeze base model layers
    base_model.trainable = False
    
    # Build the complete model
    inputs = layers.Input(shape=input_shape)
    x = xception_preprocess(inputs)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = models.Model(inputs, outputs)
    return model, base_model


In [6]:
def setup_callbacks(model_name):
    """Setup callbacks for training"""
    return [
        EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True,
            verbose=1
        ),
        ModelCheckpoint(
            f'best_{model_name}_model.h5',
            monitor='val_accuracy',
            save_best_only=True,
            save_weights_only=False,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=3,
            min_lr=1e-7,
            verbose=1
        )
    ]

def train_model(model, train_generator, val_generator, model_name, epochs=50):
    """Train the model with callbacks"""
    
    # Compile model
    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy', 'top_k_categorical_accuracy']
    )
    
    # Setup callbacks
    callbacks = setup_callbacks(model_name)
    
    # Train model
    print(f"Training {model_name} model...")
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=epochs,
        callbacks=callbacks,
        verbose=1
    )
    
    return history

def evaluate_model(model, test_generator, model_name):
    """Evaluate model performance"""
    
    print(f"Evaluating {model_name} model...")
    
    # Load best model
    best_model = tf.keras.models.load_model(f'best_{model_name}_model.h5')
    
    # Evaluate on test data
    test_loss, test_accuracy, test_top_k = best_model.evaluate(test_generator, verbose=0)
    
    # Get predictions
    predictions = best_model.predict(test_generator, verbose=0)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_generator.classes
    class_labels = list(test_generator.class_indices.keys())
    
    # Classification report
    report = classification_report(true_classes, predicted_classes, target_names=class_labels)
    
    results = {
        'model_name': model_name,
        'test_loss': test_loss,
        'test_accuracy': test_accuracy,
        'test_top_k_accuracy': test_top_k,
        'classification_report': report,
        'predictions': predictions,
        'predicted_classes': predicted_classes,
        'true_classes': true_classes,
        'class_labels': class_labels
    }
    
    return results, best_model


In [7]:
def plot_training_history(history, model_name):
    """Plot training history"""
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(f'{model_name} Training History', fontsize=16)
    
    # Plot training & validation accuracy
    axes[0, 0].plot(history.history['accuracy'], label='Training Accuracy', color='blue')
    axes[0, 0].plot(history.history['val_accuracy'], label='Validation Accuracy', color='red')
    axes[0, 0].set_title('Model Accuracy')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].legend()
    axes[0, 0].grid(True)
    
    # Plot training & validation loss
    axes[0, 1].plot(history.history['loss'], label='Training Loss', color='blue')
    axes[0, 1].plot(history.history['val_loss'], label='Validation Loss', color='red')
    axes[0, 1].set_title('Model Loss')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    # Plot top-k accuracy if available
    if 'top_k_categorical_accuracy' in history.history:
        axes[1, 0].plot(history.history['top_k_categorical_accuracy'], label='Training Top-K', color='blue')
        axes[1, 0].plot(history.history['val_top_k_categorical_accuracy'], label='Validation Top-K', color='red')
        axes[1, 0].set_title('Top-K Categorical Accuracy')
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('Top-K Accuracy')
        axes[1, 0].legend()
        axes[1, 0].grid(True)
    
    # Plot learning rate if available
    if 'lr' in history.history:
        axes[1, 1].plot(history.history['lr'], label='Learning Rate', color='green')
        axes[1, 1].set_title('Learning Rate')
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].set_ylabel('Learning Rate')
        axes[1, 1].legend()
        axes[1, 1].grid(True)
    
    plt.tight_layout()
    plt.savefig(f'{model_name}_training_history.png', dpi=300, bbox_inches='tight')
    plt.show()

def plot_confusion_matrix(true_classes, predicted_classes, class_labels, model_name):
    """Plot confusion matrix"""
    
    cm = confusion_matrix(true_classes, predicted_classes)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_labels, yticklabels=class_labels)
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.savefig(f'{model_name}_confusion_matrix.png', dpi=300, bbox_inches='tight')
    plt.show()

def compare_models(results_list):
    """Compare multiple models performance"""
    
    comparison_data = []
    for result in results_list:
        comparison_data.append({
            'Model': result['model_name'],
            'Test Accuracy': f"{result['test_accuracy']:.4f}",
            'Test Loss': f"{result['test_loss']:.4f}",
            'Top-K Accuracy': f"{result['test_top_k_accuracy']:.4f}"
        })
    
    df = pd.DataFrame(comparison_data)
    print("\n" + "="*60)
    print("MODEL COMPARISON RESULTS")
    print("="*60)
    print(df.to_string(index=False))
    print("="*60)
    
    return df


In [ ]:
def run_complete_pipeline():
    """Run the complete pipeline for all three models"""
    
    print("Starting Complete Transfer Learning Pipeline...")
    print("="*70)
    
    # Store results
    all_results = []
    all_histories = []
    
    # VGG19 Model
    print("\n1. TRAINING VGG19 MODEL")
    print("-" * 50)
    
    # Create data generators for VGG19
    vgg19_train_gen, vgg19_val_gen, vgg19_test_gen = create_data_generators_vgg19(
        train_dir, val_dir, test_dir, batch_size
    )
    
    # Get number of classes
    num_classes = vgg19_train_gen.num_classes
    print(f"Number of classes: {num_classes}")
    print(f"Training samples: {vgg19_train_gen.samples}")
    print(f"Validation samples: {vgg19_val_gen.samples}")
    print(f"Test samples: {vgg19_test_gen.samples}")
    
    # Create and train VGG19 model
    vgg19_model, vgg19_base = create_vgg19_model(num_classes)
    vgg19_model.summary()
    
    vgg19_history = train_model(vgg19_model, vgg19_train_gen, vgg19_val_gen, 'VGG19', epochs)
    vgg19_results, vgg19_best_model = evaluate_model(vgg19_model, vgg19_test_gen, 'VGG19')
    
    all_results.append(vgg19_results)
    all_histories.append(('VGG19', vgg19_history))
    
    # Plot VGG19 results
    plot_training_history(vgg19_history, 'VGG19')
    plot_confusion_matrix(vgg19_results['true_classes'], vgg19_results['predicted_classes'], 
                         vgg19_results['class_labels'], 'VGG19')
    
    print(f"\nVGG19 Results:")
    print(f"Test Accuracy: {vgg19_results['test_accuracy']:.4f}")
    print(f"Test Loss: {vgg19_results['test_loss']:.4f}")
    print("\nClassification Report:")
    print(vgg19_results['classification_report'])
    
    # ResNet50 Model
    print("\n2. TRAINING RESNET50 MODEL")
    print("-" * 50)
    
    # Create data generators for ResNet50
    resnet_train_gen, resnet_val_gen, resnet_test_gen = create_data_generators_resnet50(
        train_dir, val_dir, test_dir, batch_size
    )
    
    # Create and train ResNet50 model
    resnet50_model, resnet50_base = create_resnet50_model(num_classes)
    resnet50_model.summary()
    
    resnet50_history = train_model(resnet50_model, resnet_train_gen, resnet_val_gen, 'ResNet50', epochs)
    resnet50_results, resnet50_best_model = evaluate_model(resnet50_model, resnet_test_gen, 'ResNet50')
    
    all_results.append(resnet50_results)
    all_histories.append(('ResNet50', resnet50_history))
    
    # Plot ResNet50 results
    plot_training_history(resnet50_history, 'ResNet50')
    plot_confusion_matrix(resnet50_results['true_classes'], resnet50_results['predicted_classes'], 
                         resnet50_results['class_labels'], 'ResNet50')
    
    print(f"\nResNet50 Results:")
    print(f"Test Accuracy: {resnet50_results['test_accuracy']:.4f}")
    print(f"Test Loss: {resnet50_results['test_loss']:.4f}")
    print("\nClassification Report:")
    print(resnet50_results['classification_report'])
    
    # Xception Model
    print("\n3. TRAINING XCEPTION MODEL")
    print("-" * 50)
    
    # Create data generators for Xception
    xception_train_gen, xception_val_gen, xception_test_gen = create_data_generators_xception(
        train_dir, val_dir, test_dir, batch_size
    )
    
    # Create and train Xception model
    xception_model, xception_base = create_xception_model(num_classes)
    xception_model.summary()
    
    xception_history = train_model(xception_model, xception_train_gen, xception_val_gen, 'Xception', epochs)
    xception_results, xception_best_model = evaluate_model(xception_model, xception_test_gen, 'Xception')
    
    all_results.append(xception_results)
    all_histories.append(('Xception', xception_history))
    
    # Plot Xception results
    plot_training_history(xception_history, 'Xception')
    plot_confusion_matrix(xception_results['true_classes'], xception_results['predicted_classes'], 
                         xception_results['class_labels'], 'Xception')
    
    print(f"\nXception Results:")
    print(f"Test Accuracy: {xception_results['test_accuracy']:.4f}")
    print(f"Test Loss: {xception_results['test_loss']:.4f}")
    print("\nClassification Report:")
    print(xception_results['classification_report'])
    
    # Compare all models
    print("\n4. MODEL COMPARISON")
    print("-" * 50)
    comparison_df = compare_models(all_results)
    
    # Plot comparison
    models = [result['model_name'] for result in all_results]
    accuracies = [result['test_accuracy'] for result in all_results]
    
    plt.figure(figsize=(10, 6))
    bars = plt.bar(models, accuracies, color=['blue', 'green', 'red'], alpha=0.7)
    plt.title('Model Accuracy Comparison')
    plt.ylabel('Test Accuracy')
    plt.ylim(0, 1)
    
    # Add value labels on bars
    for bar, acc in zip(bars, accuracies):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                f'{acc:.4f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.savefig('model_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return all_results, all_histories, comparison_df

# Alternative function to run individual models
def run_individual_model(model_type='VGG19'):
    """Run individual model pipeline"""
    
    if model_type.upper() == 'VGG19':
        train_gen, val_gen, test_gen = create_data_generators_vgg19(train_dir, val_dir, test_dir, batch_size)
        model, base_model = create_vgg19_model(train_gen.num_classes)
    elif model_type.upper() == 'RESNET50':
        train_gen, val_gen, test_gen = create_data_generators_resnet50(train_dir, val_dir, test_dir, batch_size)
        model, base_model = create_resnet50_model(train_gen.num_classes)
    elif model_type.upper() == 'XCEPTION':
        train_gen, val_gen, test_gen = create_data_generators_xception(train_dir, val_dir, test_dir, batch_size)
        model, base_model = create_xception_model(train_gen.num_classes)
    else:
        raise ValueError("model_type must be 'VGG19', 'ResNet50', or 'Xception'")
    
    print(f"Training {model_type} model...")
    model.summary()
    
    # Train model
    history = train_model(model, train_gen, val_gen, model_type, epochs)
    
    # Evaluate model
    results, best_model = evaluate_model(model, test_gen, model_type)
    
    # Plot results
    plot_training_history(history, model_type)
    plot_confusion_matrix(results['true_classes'], results['predicted_classes'], 
                         results['class_labels'], model_type)
    
    print(f"\n{model_type} Results:")
    print(f"Test Accuracy: {results['test_accuracy']:.4f}")
    print(f"Test Loss: {results['test_loss']:.4f}")
    print("\nClassification Report:")
    print(results['classification_report'])
    
    return results, history, best_model

# Example usage
if __name__ == "__main__":
    # Run complete pipeline for all models
    results, histories, comparison = run_complete_pipeline()
    
    # Or run individual model
    # vgg19_result, vgg19_hist, vgg19_model = run_individual_model('VGG19')
    # resnet50_result, resnet50_hist, resnet50_model = run_individual_model('ResNet50')
    # xception_result, xception_hist, xception_model = run_individual_model('Xception')


Starting Complete Transfer Learning Pipeline...

1. TRAINING VGG19 MODEL
--------------------------------------------------
Found 2472 images belonging to 1 classes.
Found 396 images belonging to 1 classes.
Found 412 images belonging to 1 classes.
Number of classes: 1
Training samples: 2472
Validation samples: 396
Test samples: 412


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item (GetItem)  │ (None, 224, 224)  │          0 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_1          │ (None, 224, 224)  │          0 │ input_layer_1[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_2          │ (None, 224, 224)  │          0 │ input_layer_1[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack (Stack)       │ (None, 224, 224,  │          0 │ get_item[0][0],   │
│                     │ 3)                │            │ get_item_1[0][0], │
│                     │                   │            │ get_item_2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 224, 224,  │          0 │ stack[0][0]       │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vgg19 (Functional)  │ (None, 7, 7, 512) │ 20,024,384 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 512)       │          0 │ vgg19[0][0]       │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 512)       │      2,048 │ global_average_p… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 512)       │    262,656 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 512)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 256)       │    131,328 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 256)       │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │        257 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 20,420,673 (77.90 MB)

 Trainable params: 395,265 (1.51 MB)

...

c:\Users\Refat Pasha\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50


c:\Users\Refat Pasha\anaconda3\Lib\site-packages\keras\src\ops\nn.py:944: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
c:\Users\Refat Pasha\anaconda3\Lib\site-packages\keras\src\losses\losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00 - top_k_categorical_accuracy: 1.0000
Epoch 1: val_accuracy improved from -inf to 1.00000, saving model to best_VGG19_model.h5


78/78 ━━━━━━━━━━━━━━━━━━━━ 252s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00 - top_k_categorical_accuracy: 1.0000 - val_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_top_k_categorical_accuracy: 1.0000 - learning_rate: 0.0010
Epoch 2/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - top_k_categorical_accuracy: 1.0000
Epoch 2: val_accuracy did not improve from 1.00000
78/78 ━━━━━━━━━━━━━━━━━━━━ 224s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00 - top_k_categorical_accuracy: 1.0000 - val_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_top_k_categorical_accuracy: 1.0000 - learning_rate: 0.0010
Epoch 3/50
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00 - top_k_categorical_accuracy: 1.0000
Epoch 3: val_accuracy did not improve from 1.00000
78/78 ━━━━━━━━━━━━━━━━━━━━ 230s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00 - top_k_categorical_accuracy: 1.0000 - val_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_top_k_categorical_accuracy: 1.0

In [ ]:
def fine_tune_model(model, base_model, train_generator, val_generator, model_name, 
                   fine_tune_at=100, learning_rate=1e-5, fine_tune_epochs=10):
    """Fine-tune the pre-trained model by unfreezing some layers"""
    
    # Unfreeze the top layers of the base model
    base_model.trainable = True
    
    # Fine-tune from this layer onwards
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False
    
    # Recompile with lower learning rate
    model.compile(
        optimizer=optimizers.Adam(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy', 'top_k_categorical_accuracy']
    )
    
    # Setup callbacks for fine-tuning
    callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True,
            verbose=1
        ),
        ModelCheckpoint(
            f'fine_tuned_{model_name}_model.h5',
            monitor='val_accuracy',
            save_best_only=True,
            save_weights_only=False,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=2,
            min_lr=1e-8,
            verbose=1
        )
    ]
    
    print(f"Fine-tuning {model_name} model...")
    
    # Continue training with fine-tuning
    history_fine = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=fine_tune_epochs,
        callbacks=callbacks,
        verbose=1
    )
    
    return history_fine


Epoch 51/60
 2/62 ━━━━━━━━━━━━━━━━━━━━ 2:27 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - top_k_categorical_accuracy: 1.0000

KeyboardInterrupt: 

In [ ]:
# Function to make predictions on new images
def predict_image(model, image_path, class_names):
    """
    Predict class for a single image
    """
    from tensorflow.keras.preprocessing import image
    
    # Load and preprocess image
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    
    # Make prediction
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions[0])
    confidence = predictions[0][predicted_class]
    
    print(f"Predicted class: {class_names[predicted_class]}")
    print(f"Confidence: {confidence:.4f}")
    
    return predicted_class, confidence

# Get class names
class_names = list(train_generator.class_indices.keys())

# Example prediction (replace with your image path)
# predicted_class, confidence = predict_image(best_model, 'path/to/your/image.jpg', class_names)
